In [ ]:
pip install transformers tensorflow requests matplotlib

In [ ]:
import requests
import json
import numpy as np
from transformers import pipeline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# --------------------------------------
# Amazon API Data Fetching
# --------------------------------------
def fetch_campaign_data(campaign_id, start_date, end_date, auth_token):
    """
    Fetch data from Amazon Advertising API for a specific ad campaign.
    """
    endpoint = f'https://advertising-api.amazon.com/v2/sp/campaigns/{campaign_id}/adGroups'
    headers = {
        'Authorization': f'Bearer {auth_token}',
        'Content-Type': 'application/json',
    }
    params = {
        'startDate': start_date,
        'endDate': end_date,
    }
    response = requests.get(endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

# --------------------------------------
# BERTweet Keyword Extraction
# --------------------------------------
def extract_keywords(customer_search_terms):
    """
    Extract keywords using 'finiteautomata/bertweet-base-sentiment-analysis' model.
    Only keep terms with positive or neutral sentiment.
    """
    model = pipeline('sentiment-analysis', model='finiteautomata/bertweet-base-sentiment-analysis')
    extracted_keywords = []
    for term in customer_search_terms:
        sentiment = model(term)[0]
        # Keep only terms with positive or neutral sentiment
        if sentiment['label'] in ['POS', 'NEU']:
            extracted_keywords.append(term)
    return extracted_keywords

# --------------------------------------
# CNN for Keyword Classification
# --------------------------------------
def build_cnn_model(input_shape):
    """
    Build a CNN model for classifying keywords into profitable, non-profitable, and neutral.
    """
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(64, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))  # 3 output classes: Profitable, Non-profitable, Neutral
    return model

def classify_keywords_with_cnn(keywords, performance_data, cnn_model):
    """
    Classify keywords using CNN into Profitable, Non-profitable, and Neutral categories.
    """
    keyword_classifications = {}
    for keyword in keywords:
        features = np.array(performance_data[keyword])  # Get performance data for the keyword
        features_reshaped = features.reshape((1, len(features), 1))  # Reshape for CNN input
        
        prediction = cnn_model.predict(features_reshaped)
        class_label = ['Profitable', 'Non-profitable', 'Neutral'][prediction.argmax()]
        keyword_classifications[keyword] = class_label
    return keyword_classifications

# --------------------------------------
# Visual Representation (Pie Chart)
# --------------------------------------
def plot_pie_chart(profitable, non_profitable, neutral):
    """
    Plot a pie chart of keyword classifications.
    """
    labels = 'Profitable', 'Non-profitable', 'Neutral'
    sizes = [len(profitable), len(non_profitable), len(neutral)]
    
    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures the pie is drawn as a circle.
    
    plt.show()

# --------------------------------------
# Main Pipeline for Fetching Data, Keyword Extraction, Classification, and Visualization
# --------------------------------------
def amazon_campaign_optimizer(campaign_id, start_date, end_date, auth_token):
    """
    Complete workflow for optimizing an Amazon ad campaign.
    - Fetches data via Amazon API
    - Extracts keywords with BERTweet
    - Classifies keywords using CNN model
    - Visualizes results
    """
    # 1. Fetch Campaign Data
    campaign_data = fetch_campaign_data(campaign_id, start_date, end_date, auth_token)
    
    if not campaign_data:
        return "No data available."

    # 2. Extract Keywords from Customer Search Terms
    customer_search_terms = [ad['searchTerm'] for ad in campaign_data['adGroups']]  # Example extraction
    extracted_keywords = extract_keywords(customer_search_terms)

    # 3. Prepare Performance Data for Classification
    # Simulate performance data (Clicks, CPC, etc.) for CNN input
    performance_data = {
        keyword: [np.random.randint(50, 500), np.random.uniform(0.5, 3.0), np.random.randint(100, 5000)] for keyword in extracted_keywords
    }

    # 4. Build and Train CNN Model
    input_shape = (len(performance_data[extracted_keywords[0]]), 1)  # Define input shape based on performance data
    cnn_model = build_cnn_model(input_shape)
    
    # Simulate training data (in practice, you'd need labeled data here)
    X_train = np.array(list(performance_data.values()))
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = tf.keras.utils.to_categorical(np.random.randint(3, size=(X_train.shape[0],)))  # Simulate labels
    
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    cnn_model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2)

    # 5. Classify Extracted Keywords
    classified_keywords = classify_keywords_with_cnn(extracted_keywords, performance_data, cnn_model)

    # 6. Present Results
    profitable = [k for k, v in classified_keywords.items() if v == 'Profitable']
    non_profitable = [k for k, v in classified_keywords.items() if v == 'Non-profitable']
    neutral = [k for k, v in classified_keywords.items() if v == 'Neutral']
    
    print("Profitable Keywords:", profitable)
    print("Non-profitable Keywords:", non_profitable)
    print("Neutral Keywords:", neutral)

    # 7. Visualize Classification Results
    plot_pie_chart(profitable, non_profitable, neutral)

# --------------------------------------
# Example Usage
# --------------------------------------
campaign_id = '12345'
start_date = '2024-01-01'
end_date = '2024-01-31'
auth_token = 'YOUR_AUTH_TOKEN'

amazon_campaign_optimizer(campaign_id, start_date, end_date, auth_token)
